In [97]:
import finite_differences.finite_diff_formulas as f
import importlib as i
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
i.reload(f)
import test_fd_methods as t
i.reload(t)
import test_implementation as oi
i.reload(oi)

<module 'test_implementation' from '/Users/manuelpanora/Python/Finite-Difference-Option-Pricer/test_implementation.py'>

# Option Pricing with the Crank-Nicolson Method

## Overview
This notebook demonstrates pricing European call options using methods of PDE solutions. The two main methods explored are the Crank-Nicolson method and forward finite differnces. The goal of this project is to price exotic options using similar numerical methods. 

**Key Features**:
- Finite difference method with dynamic grid sizing.
- Sparse matrix solving for efficiency.
- Comparison to Black-Scholes analytical solution.
- Rigorous Testing to improve stability and accuracy

The parameters below are used for intial method testing. The analytical method and the naive finite difference method are displayed first.

In [55]:
TRADING_DAYS = 252
K = 95
S = 100
sigma = .2
r = .05
tfinal = 21 / TRADING_DAYS  

#n is discrete time steps, m is assset price steps
n = 16
m = 16        

bs_price = f.black_scholes_call(S, K, tfinal, r, sigma)
print("European Call Option Price Analytical:", bs_price)

European Call Option Price Analytical: 5.898921658718436


In [56]:
price_fd = f.finite_difference_call(n, m, K, tfinal, S, sigma, r)
price_cn = f.price_derivative_cn(n, m, K, r, tfinal, S, sigma)


With the completed implementations of both methods, we can display the results. We observe that given our starting conditions, both numerical methods get within one digit of accuracy.

In [57]:
print("When given", m, "pricesteps and", n, "timesteps:")
print("European Call Option Price Analytical:", bs_price)
print("European Call Option Price Finite Dif:", price_fd)
print("European Call Option Price Crank-Nico:", price_cn)

When given 16 pricesteps and 16 timesteps:
European Call Option Price Analytical: 5.898921658718436
European Call Option Price Finite Dif: 5.876594250948936
European Call Option Price Crank-Nico: 5.209677960940858


We continue to analyze the convergence of the solution. Analyzing and finding the source of innacuracy is the primary focus of attention. Using an online source as a method of validation, we compute the option price for varying asset and time step values. As the rsults show, when we simply increase the number of timesteps or asset price steps, it is clear that increasing the asset price steps influences the convergence of the solution more. 

In [69]:
cn_errors, oi_errors, cn_values = t.test_convergence_all(K, r, tfinal, S, sigma, (5,4), f.price_derivative_cn, oi.crank_nicholson)
print(cn_errors)
print(oi_errors)

[[4.54694904 2.28848494 0.66416774 0.6935946 ]
 [4.61812658 2.17437862 0.48025128 0.6132524 ]
 [4.66547576 2.09855622 0.36155887 0.57024542]
 [4.68267334 2.07103364 0.31913605 0.55573814]
 [4.68747074 2.06335759 0.30736561 0.5517887 ]]
[[3.0410532  1.76629646 0.38538074 0.4782477 ]
 [3.04105301 1.76629409 0.38538928 0.47657931]
 [3.04105295 1.76629339 0.38539182 0.47610064]
 [3.04105294 1.76629331 0.38539211 0.47604569]
 [3.04105294 1.7662933  0.38539213 0.47604157]]


We will focus on creating the optimal grid that results in the smallest errors. The test is stil conducted using various asset price step sizes. Empircal testing shows that 3 standard deviations used to calculate the boundary values for the asset price grid are sufficient and yield the best numerical results in one particular case. Testing is provided below. With this method, we have narrowed down the range of standard deviations to (2,5) as these ranges provide best convergence and numerical stability. For additional testing, we expriment with varying starting values. (Best combination so far (4,2,3))

In [61]:
def dynamic_bounds(K, sigma, T, n_std=5):
    """
    Generate bounds based on lognormal distribution percentiles.
    """
    S_min = K * np.exp(-n_std * sigma * np.sqrt(T))
    S_max = K * np.exp(n_std * sigma * np.sqrt(T))
    return (S_min, S_max)

In [87]:
#Static boundaries: 
s_vals = [(K/3, K*2), (K/4, K*3)]

#Dynamic boundaries:
d_vals = np.array([dynamic_bounds(K, sigma, tfinal, i) for i in range(2,5)])

cn_errors, oi_errors = t.test_grid_asset_boundaries(f.price_derivative_cn, oi.crank_nicholson, K, r, tfinal, S, sigma, (6,5), d_vals)

for i in range(len(cn_errors)):
    print(d_vals[i])
    print(cn_errors[i])

[ 93.87129414 106.52883921]
[[0.77532033 0.3160485  0.42068274 0.71793079 1.30913512]
 [0.6251313  0.15707062 0.24888416 0.55340612 1.19259979]
 [0.52990449 0.05566836 0.1637352  0.551399   1.25647   ]
 [0.49620838 0.01983332 0.13348973 0.54208321 1.27426569]
 [0.48689165 0.00993453 0.12515464 0.53943367 1.27650956]
 [0.48450045 0.00739469 0.12301753 0.53875701 1.27703918]]
[ 90.94926797 109.95140724]
[[7.96495992e-01 2.71301042e-01 3.84692139e-01 6.38280558e-01
  1.12703224e+00]
 [6.73389897e-01 1.27496312e-01 2.25931402e-01 4.28716189e-01
  1.02374892e+00]
 [5.93299255e-01 3.38308652e-02 1.37096832e-01 4.22303521e-01
  1.09799822e+00]
 [5.64560812e-01 2.28199627e-04 1.05681339e-01 4.09505523e-01
  1.11739413e+00]
 [5.56577120e-01 9.10501084e-03 9.69965742e-02 4.06010153e-01
  1.12074835e+00]
 [5.54525393e-01 1.15033987e-02 9.47676802e-02 4.05117630e-01
  1.12160473e+00]]
[ 88.11819864 113.48393583]
[[0.69705782 0.20390168 0.37664957 0.53507363 0.99487978]
 [0.59103988 0.07425545 0.22

Our next step is to test various starting conditions to see which these work best for. Our testing shows that this method works best for low volatility, short time to maturity. Generally speaking, 3 tends to produce the best results out of any even though some of the boundary conditions are not as good as others. 

In [100]:
# Test cases: K, σ, T, S0
test_cases = [
    # Low volatility, short maturity
    {"K": 100, "sigma": 0.1, "T": 0.1, "S0": 100, "type" : "Low volatility, short maturity"},
    # High volatility, long maturity
    {"K": 100, "sigma": 0.5, "T": 5.0, "S0": 100, "type" : "High volatility, long maturity"},
    # Extreme moneyness (Strike price is significantly higher/lower than current stock price )
    {"K": 100, "sigma": 0.3, "T": 1.0, "S0": 50, "type" : "OTM"},   # OTM call
    {"K": 100, "sigma": 0.3, "T": 1.0, "S0": 150, "type" : "ITM"},  # ITM call
    # Small/large strikes
    {"K": 10,  "sigma": 0.2, "T": 1.0, "S0": 10, "type" : "Small strike"},
    {"K": 500, "sigma": 0.4, "T": 1.0, "S0": 500, "type" : "Large strike"},
]

for case in test_cases:
    K = case["K"]
    sigma = case["sigma"]
    tfinal = case["T"]
    S0 = case["S0"]
    
    d_vals = np.array([dynamic_bounds(K, sigma, tfinal, i) for i in range(1,10)])

    cn_errors, oi_errors = t.test_grid_asset_boundaries(f.price_derivative_cn, oi.crank_nicholson, K, r, tfinal, S0, sigma, (6,5), d_vals)
    print(case["type"])
    for i in range(len(cn_errors)):
        print(d_vals[i])
        print(cn_errors[i])
    
    print()

Low volatility, short maturity
[ 96.88719943 103.2128089 ]
[[0.55325651 0.34008697 0.49757311 0.87305758 1.45282253]
 [0.40080459 0.19567773 0.29651635 0.64061555 1.23549615]
 [0.31740326 0.12536122 0.25333731 0.66970601 1.27950822]
 [0.29013747 0.10317816 0.23671123 0.67580921 1.2935277 ]
 [0.28279737 0.09727827 0.23233512 0.67488472 1.29528356]
 [0.28092719 0.09578004 0.23122704 0.6746554  1.29538829]]
[ 93.87129414 106.52883921]
[[0.77532033 0.3160485  0.42068274 0.71793079 1.30913512]
 [0.6251313  0.15707062 0.24888416 0.55340612 1.19259979]
 [0.52990449 0.05566836 0.1637352  0.551399   1.25647   ]
 [0.49620838 0.01983332 0.13348973 0.54208321 1.27426569]
 [0.48689165 0.00993453 0.12515464 0.53943367 1.27650956]
 [0.48450045 0.00739469 0.12301753 0.53875701 1.27703918]]
[ 90.94926797 109.95140724]
[[7.96495992e-01 2.71301042e-01 3.84692139e-01 6.38280558e-01
  1.12703224e+00]
 [6.73389897e-01 1.27496312e-01 2.25931402e-01 4.28716189e-01
  1.02374892e+00]
 [5.93299255e-01 3.38308652